### Setting Up the Environment and Loading Data 

First, the necessary libraries were imported. We'll use ```pandas``` for data manipulation and ```pulp``` for formulating and solving the optimization problem.

In [1]:
from pulp import *
import pandas as pd

Next, we load the data from an Excel file (```diet.xls```). Viewing the first few rows of ```df``` shows the structure of the data, the file contains nutritional information, such as calories, protein and vitamins for different foods. 

In [2]:
df = pd.read_excel("data/diet.xls")
df.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [3]:
df.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


Toward the end of the data, there are additional rows specifying the minimum and maximum daily nutritional requirements.

### Extracting Nutritional Requirements
The next step is to extract the rows containing these nutritional requirements. By inspecting the dataframe, we observe that rows 65 to 67 contain this information. We select these rows and clean up the DataFrame by removing any irrelevant columns.

In [4]:
requirement_df = df.iloc[65:67]
requirement_df = requirement_df.drop(columns = ['Foods','Price/ Serving']).reset_index(drop= True)
requirement_df.head()

,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
1,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


The resulting dataframe, ```requirement_df``` contains the minimum and maximum values for daily intake of each nutrient, which will become our constraints in the optimization problem.

### Organizing Food Data
Next, we extract rows 0 to 64, which contain the food items and their respective nutrient values. To make the data easier to work with, the dataframe is transposed so that each column corresponds to a specific food and each row to a nutrient. 

In [5]:
food_df = df.iloc[0:64]
food_df = food_df.transpose()
food_df.columns = food_df.iloc[0]
food_df = food_df.drop(index='Foods')
food_df.columns.name = None

food_df


,Frozen Broccoli,"Carrots,Raw","Celery, Raw",Frozen Corn,"Lettuce,Iceberg,Raw","Peppers, Sweet, Raw","Potatoes, Baked",Tofu,Roasted Chicken,Spaghetti W/ Sauce,...,Pretzels,Tortilla Chip,Chicknoodl Soup,Splt Pea&Hamsoup,Vegetbeef Soup,Neweng Clamchwd,Tomato Soup,"New E Clamchwd,W/Mlk","Crm Mshrm Soup,W/Mlk","Beanbacn Soup,W/Watr"
Price/ Serving,0.16,0.07,0.04,0.18,0.02,0.53,0.06,0.31,0.84,0.78,...,0.12,0.19,0.39,0.67,0.71,0.75,0.39,0.99,0.65,0.67
Serving Size,10 Oz Pkg,1/2 Cup Shredded,1 Stalk,1/2 Cup,1 Leaf,1 Pepper,1/2 Cup,1/4 block,1 lb chicken,1 1/2 Cup,...,1 Oz,1 Oz,1 C (8 Fl Oz),1 C (8 Fl Oz),1 C (8 Fl Oz),1 C (8 Fl Oz),1 C (8 Fl Oz),1 C (8 Fl Oz),1 C (8 Fl Oz),1 C (8 Fl Oz)
Calories,73.8,23.7,6.4,72.2,2.6,20.0,171.5,88.2,277.4,358.2,...,108.0,142.0,150.1,184.8,158.1,175.7,170.7,163.7,203.4,172.0
Cholesterol mg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,129.9,0.0,...,0.0,0.0,12.3,7.2,10.0,10.0,0.0,22.3,19.8,2.5
Total_Fat g,0.8,0.1,0.1,0.6,0.0,0.1,0.2,5.5,10.8,12.3,...,1.0,7.4,4.6,4.0,3.8,5.0,3.8,6.6,13.6,5.9
Sodium mg,68.2,19.2,34.8,2.5,1.8,1.5,15.2,8.1,125.6,1237.1,...,486.2,149.7,1862.2,964.8,1915.1,1864.9,1744.4,992.0,1076.3,951.3
Carbohydrates g,13.6,5.6,1.5,17.1,0.4,4.8,39.9,2.2,0.0,58.3,...,22.5,17.8,18.7,26.8,20.4,21.8,33.2,16.6,15.0,22.8
Dietary_Fiber g,8.5,1.6,0.7,2.0,0.3,1.3,3.2,1.4,0.0,11.6,...,0.9,1.8,1.5,4.1,4.0,1.5,1.0,1.5,0.5,8.6
Protein g,8.0,0.6,0.3,2.5,0.2,0.7,3.7,9.4,42.2,8.2,...,2.6,2.0,7.9,11.1,11.2,10.9,4.1,9.5,6.1,7.9
Vit_A IU,5867.4,15471.0,53.6,106.6,66.0,467.7,0.0,98.6,77.4,3055.2,...,0.0,55.6,1308.7,4872.0,3785.1,20.1,1393.0,163.7,153.8,888.0


The result is a tidy dataframe where each column represents a food item, and each row represents a specific nutrient. 

### Preparing Data for Optimization
Since ```pulp``` functions take lists and dictionaries as input, we need to convert the dataframe into the appropraite data structures. We create a list of all food items and a dictionary (```row_dict```) that maps each nutrient to a dictionary of its values for each food item. 

In [6]:
Foods = list(food_df.columns)

row_dict = {}

for index, row in food_df.iterrows():
    row_dict[index] = dict(row)
    

#print(f"Foods: {Foods}")

print(row_dict)

{'Price/ Serving': {'Frozen Broccoli': 0.16, 'Carrots,Raw': 0.07, 'Celery, Raw': 0.04, 'Frozen Corn': 0.18, 'Lettuce,Iceberg,Raw': 0.02, 'Peppers, Sweet, Raw': 0.53, 'Potatoes, Baked': 0.06, 'Tofu': 0.31, 'Roasted Chicken': 0.84, 'Spaghetti W/ Sauce': 0.78, 'Tomato,Red,Ripe,Raw': 0.27, 'Apple,Raw,W/Skin': 0.24, 'Banana': 0.15, 'Grapes': 0.32, 'Kiwifruit,Raw,Fresh': 0.49, 'Oranges': 0.15, 'Bagels': 0.16, 'Wheat Bread': 0.05, 'White Bread': 0.06, 'Oatmeal Cookies': 0.09, 'Apple Pie': 0.16, 'Chocolate Chip Cookies': 0.03, 'Butter,Regular': 0.05, 'Cheddar Cheese': 0.25, '3.3% Fat,Whole Milk': 0.16, '2% Lowfat Milk': 0.23, 'Skim Milk': 0.13, 'Poached Eggs': 0.08, 'Scrambled Eggs': 0.11, 'Bologna,Turkey': 0.15, 'Frankfurter, Beef': 0.27, 'Ham,Sliced,Extralean': 0.33, 'Kielbasa,Prk': 0.15, "Cap'N Crunch": 0.31, 'Cheerios': 0.28, "Corn Flks, Kellogg'S": 0.28, "Raisin Brn, Kellg'S": 0.34, 'Rice Krispies': 0.32, 'Special K': 0.38, 'Oatmeal': 0.82, 'Malt-O-Meal,Choc': 0.52, 'Pizza W/Pepperoni': 0

### Defining the Optimization Problem
We start by creating a Lp problem, specifying that it’s a minimization problem (since we want to minimize the total cost of food) and name it "The_Diet_Problem."

In [7]:
# Create the 'prob' variable to contain the problem data
prob =LpProblem("The_Diet_Problem", LpMinimize)

Next, we define variables representing the quantity of each food in the diet. ```food_vars``` are continuous and non-negative, representing the number of servings for each food.

In [8]:
food_vars = LpVariable.dicts("Ingr", Foods, 0)

To minimize the cost, we set up an objective function that sums the cost per serving of each food multiplied by the servings chosen (```food_vars```). 

In [9]:
# The objective function is added to 'prob' first
prob += (
    lpSum([row_dict['Price/ Serving'][i] * food_vars[i] for i in Foods]),
    "Total Cost of Ingredients",
)



### Constraints: Meeting Nutritional Requirements
The constraint requires that the selected foods meet minimum and maximum values for each nutrient. A loop is set up to iterate through each nutrient in ```row_dict``` and add two constraints for each: one to ensure that nutrient intake is above the minimum and another to ensure it is below the maximum.

In [10]:
# The constraints are added to 'prob'
del row_dict['Price/ Serving']
del row_dict['Serving Size']

for key, value in row_dict.items():
    
    prob += (
    lpSum([value[i] * food_vars[i] for i in Foods]) >= requirement_df[key].iloc[0],
    f"{key} minimum",
)
    prob += (
    lpSum([value[i] * food_vars[i] for i in Foods]) <= requirement_df[key].iloc[1],
    f"{key} maximum",
)    
    

### Solving the Diet Optimization Problem
After setting up the objective and constraints, we can solve the problem using PuLP's default solver. The solution will provide the optimal servings of each food that minimize cost while meeting the nutritional constraints.

In [11]:
prob.solve()

1

In [12]:
for v in prob.variables():
    if v.varValue != 0:
        print(v.name, "=", v.varValue)
print("Total Cost of Foods = ", pulp.value(prob.objective))

Ingr_Celery,_Raw = 52.64371
Ingr_Frozen_Broccoli = 0.25960653
Ingr_Lettuce,Iceberg,Raw = 63.988506
Ingr_Oranges = 2.2929389
Ingr_Poached_Eggs = 0.14184397
Ingr_Popcorn,Air_Popped = 13.869322
Total Cost of Foods =  4.337116797399999


Foods with non-zero values are included in the diet, and the total cost of the diet is 4.34.

### Part 2: Adding New Constraints
Let's add some new constraints to make our optimization problem more interesting: 
- If a food item is chosen, a minimum portion of 0.1 serving must be included..
- Only one of celery and frozen broccoli can be selected at the same time. 
- At least 3 kinds of high protein food must be selected.

Meeting these constraints require an additional binary variable, ```food_select_var```,  for each food, indicating whether the food is included in the diet (1 if included, 0 otherwise).

In [13]:
additional_prob = prob.copy()
food_select_var = LpVariable.dicts("food_select",Foods,0,1,LpBinary)

#### Serving Size Requirement
To meet the first additional requirement, we add constraints to ensure that if a food is selected (```food_select_var[i] == 1```), at least 0.1 servings must be chosen. We also add a constraint to ensure that if any amount of food is chosen, it is marked as selected (```food_select_var[i] == 1```).

In [14]:

for i in Foods:
    additional_prob += food_vars[i] >= 0.1* food_select_var[i] #If a food is selected, a minimum of 0.1 servings must be included.
    additional_prob += food_select_var[i] >= 0.00001* food_vars[i] #If any amount of a food is included, the food must be marked as selected (marked as 1).

#### Celery vs Broccoli
To make sure that not both celery and frozen broccoli are chosen, we specify that the sum of their ```food_select_var``` cannot be larger than 1. 

In [15]:
additional_prob += food_select_var['Frozen Broccoli'] + food_select_var['Celery, Raw'] <= 1

#### Protein Variety Requirement
To make sure there are varieties in protein sources, we add a constraint requiring at least three foods with high protein (≥ 8g per serving). We identify these foods by filtering ```food_df``` and require that at least three of them are included.

In [16]:
protein_df = food_df.loc[:, food_df.loc['Protein g'] >= 8]
protein_list = list(protein_df.columns)
additional_prob += lpSum([food_select_var[protein] for protein in protein_list]) >= 3

The solution output the selected foods, quantities, and total cost.

In [17]:
additional_prob.solve()

for v in additional_prob.variables():
    if v.varValue != 0:
        print(v.name, "=", v.varValue)

print("Total Cost of Foods = ", pulp.value(additional_prob.objective))

Ingr_3.3%_Fat,Whole_Milk = 0.1
Ingr_Celery,_Raw = 42.97197
Ingr_Lettuce,Iceberg,Raw = 83.315741
Ingr_Oranges = 3.0494389
Ingr_Peanut_Butter = 1.9128087
Ingr_Poached_Eggs = 0.12406619
Ingr_Popcorn,Air_Popped = 13.096451
Ingr_Skim_Milk = 0.1
Ingr_Tofu = 0.1
food_select_3.3%_Fat,Whole_Milk = 1.0
food_select_Celery,_Raw = 1.0
food_select_Lettuce,Iceberg,Raw = 1.0
food_select_Oranges = 1.0
food_select_Peanut_Butter = 1.0
food_select_Poached_Eggs = 1.0
food_select_Popcorn,Air_Popped = 1.0
food_select_Skim_Milk = 1.0
food_select_Tofu = 1.0
Total Cost of Foods =  4.5702893992
